In [ ]:
import os
import sys
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
sys.path.append('/home/chendian/BBCM/')

In [10]:
train_dataset = [line.strip() for line in open("/data/chendian/cleaned_findoc_samples/autodoc_train_1.6M.220425.txt", 'r')]
dev_dataset = [line.strip() for line in open("/data/chendian/cleaned_findoc_samples/autodoc_train_last6k.220425.txt", 'r')]
test_dataset = [line.strip() for line in open("/data/chendian/cleaned_findoc_samples/autodoc_test.220424.txt", 'r')]

In [45]:
train_dataset[:10]

['在观测样本中随机选择一个实体，将其替换为集合中的另一个同类型实体，从而造成一个心的反事实样本。\t在观测样本中随机选择一个实体，将其替换为集合中的另一个同类型实体，从而造成一个新的反事实样本。',
 '有一些比较新的语义搜索的前言工作，好像我们有几位同学是在做这个的，说不定能有点启发。\t有一些比较新的语义搜索的前沿工作，好像我们有几位同学是在做这个的，说不定能有点启发。',
 '老师可能还是不太明白这个算法是如何工作的，我打算画张图给他看快。\t老师可能还是不太明白这个算法是如何工作的，我打算画张图给他看看。',
 '我们宿舍的窗帘遮光性太差，太阳一照就行了。\t我们宿舍的窗帘遮光性太差，太阳一照就醒了。',
 '要是针对免费了，我明天就坐两站路去麦当劳吃。\t要是真的免费了，我明天就坐两站路去麦当劳吃。',
 '跑完步之后我贞德浑身酸疼，特别难受。\t跑完步之后我真的浑身酸疼，特别难受。',
 '要不然这也来过分了！\t要不然这也太过分了！',
 '（1）拟变更债券募集说明书的重要约订；\t（1）拟变更债券募集说明书的重要约定；',
 '“付款义务人”焦易记录良好，在过往三年所有历史交易中最长回款天数为超过其相应应收账款“预期付款日”后的6个月；\t“付款义务人”交易记录良好，在过往三年所有历史交易中最长回款天数为超过其相应应收账款“预期付款日”后的6个月；',
 '《发行人服务协议》：西指“受托人”与“登记托管机构”/“支付代理机构”签署《发行人服务协议》及对该合同的任何修改或补充。\t《发行人服务协议》：系指“受托人”与“登记托管机构”/“支付代理机构”签署《发行人服务协议》及对该合同的任何修改或补充。']

In [46]:
get_faulty_pairs(train_dataset[:10])

100%|█████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 27850.62it/s]


[('心', '新'),
 ('言', '沿'),
 ('快', '看'),
 ('行', '醒'),
 ('针对', '真的'),
 ('贞德', '真的'),
 ('来', '太'),
 ('订', '定'),
 ('焦', '交'),
 ('西', '系')]

In [14]:
from tqdm import tqdm

def get_faulty_pairs(dataset):
    pairs = []
    for line in tqdm(dataset):
        err, cor = line.split()
        err, cor = err.strip(), cor.strip()
        wrong_ids = [i for i, (_e, _c) in enumerate(zip(err, cor)) if _e != _c]

        streak = []
        for _i, w in enumerate(wrong_ids):
            streak.append(w)
            if _i != 0 and w != wrong_ids[_i-1] + 1:
                pairs.append((
                    ''.join([err[_w] for _w in streak]), 
                    ''.join([cor[_w] for _w in streak]), 
                ))
                streak = []
        else:
            if streak:
                pairs.append((
                    ''.join([err[_w] for _w in streak]), 
                    ''.join([cor[_w] for _w in streak]), 
                ))
                streak = []
    return pairs

train_pairs = get_faulty_pairs(train_dataset)
dev_pairs = get_faulty_pairs(dev_dataset)
test_pairs = get_faulty_pairs(test_dataset)

100%|█████████████████████████████████████████████████████████████████████████| 5292/5292 [00:00<00:00, 50681.01it/s]


In [8]:
from tqdm import tqdm

dev_pairs = []

for line in tqdm(dev_dataset):
    err, cor = line.split()
    err, cor = err.strip(), cor.strip()
    wrong_ids = [i for i, (_e, _c) in enumerate(zip(err, cor)) if _e != _c]
    
    streak = []
    for _i, w in enumerate(wrong_ids):
        streak.append(w)
        if _i != 0 and w != wrong_ids[_i-1] + 1:
            dev_pairs.append((
                ''.join([err[_w] for _w in streak]), 
                ''.join([cor[_w] for _w in streak]), 
            ))
            streak = []
    else:
        if streak:
            dev_pairs.append((
                ''.join([err[_w] for _w in streak]), 
                ''.join([cor[_w] for _w in streak]), 
            ))
            streak = []

100%|█████████████████████████████████████████████████████████████████████████| 6000/6000 [00:00<00:00, 42028.22it/s]


In [9]:
from tqdm import tqdm

test_pairs = []

for line in tqdm(test_dataset):
    err, cor = line.split()
    err, cor = err.strip(), cor.strip()
    wrong_ids = [i for i, (_e, _c) in enumerate(zip(err, cor)) if _e != _c]
    
    streak = []
    for _i, w in enumerate(wrong_ids):
        streak.append(w)
        if _i != 0 and w != wrong_ids[_i-1] + 1:
            test_pairs.append((
                ''.join([err[_w] for _w in streak]), 
                ''.join([cor[_w] for _w in streak]), 
            ))
            streak = []
    else:
        if streak:
            test_pairs.append((
                ''.join([err[_w] for _w in streak]), 
                ''.join([cor[_w] for _w in streak]), 
            ))
            streak = []

100%|█████████████████████████████████████████████████████████████████████████| 5292/5292 [00:00<00:00, 45036.54it/s]


In [15]:
train_pairs_set = set(train_pairs)
print(len(set(dev_pairs).intersection(train_pairs_set)) / len(dev_pairs))
print(len(set(test_pairs).intersection(train_pairs_set)) / len(test_pairs))

0.12583001328021248
0.225974930362117


In [16]:
# 混淆集生成组件
import os
import sys
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
sys.path.append('/home/chendian/BBCM/')
from bbcm.data.loaders.confusor import Confusor

/home/chendian/.conda/envs/bbcm/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
conf = Confusor(
    cand_pinyin_num=30, 
    cos_threshold=(0., 0.75), 
    method='beam', 
    token_sample_mode='sort', 
    pinyin_sample_mode='special', 
    weight=[3, 0, 0.33],   # pinyin score, word freq score
    conf_size=300, 
    debug=False
)

Use beam method.
Pinyin sampling mode: special.
Token sampling mode: sort.
Now loading pinyin2token corpus.
Loading pinyin2token_new.pkl (1070.18MB)
Now loading zi_sim_matrix.
Loading zi_sim_matrix.pkl (3.03MB)
Now loading REDscore:
Loading ziREDscore.pkl (1.93MB)
Now generating score matrix.
Now Loading word freuency data:
Loading findoc_word_frequency_score_01.pkl (19.45MB)
Loading findoc_word2_frequency_score.pkl (281.79MB)
Loading wc_word_frequency_score_01.pkl (59.35MB)
Loading wc_word2_frequency_score.pkl (2084.0MB)


In [25]:
from tqdm import tqdm

dev_rec = {}
dev_uncatched = []
for err, cor in tqdm(dev_pairs):
    cfs = conf(cor)
    if err in cfs:
        dev_rec[(err, cor)] = cfs.index(err)
    else:
        dev_uncatched.append((err, cor))
        # dev_rec[(err, cor)] = len(cfs) * 2
    
test_rec = {}
test_uncatched = []
for err, cor in tqdm(test_pairs):
    cfs = conf(cor)
    if err in cfs:
        test_rec[(err, cor)] = cfs.index(err)
    else:
        test_uncatched.append((err, cor))
        # test_rec[(err, cor)] = len(cfs) * 2

100%|████████████████████████████████████████████████████████████████████████████| 2872/2872 [06:02<00:00,  7.93it/s]


In [24]:
len(uncatched)

2567

In [30]:
len(dev_uncatched) / (len(dev_rec)+len(dev_uncatched)), sum(dev_rec.values()) / (len(dev_rec))

(0.17086546700942587, 16.16287722199256)

In [31]:
len(test_uncatched) / (len(test_rec)+len(test_uncatched)), sum(test_rec.values()) / (len(test_rec))

(0.6885964912280702, 63.1887323943662)

In [40]:
unk = 0
for err, cor in test_uncatched:
    if conf.word_freq.get(cor, -0.1) == -0.1:
        unk += 1

In [44]:
test_uncatched[:50]

[('适当', '使得'),
 ('已经', '以及'),
 ('极限', '机械'),
 ('勾', '钩'),
 ('业', '要'),
 ('园林', '原料'),
 ('个', '改'),
 ('以及', '移交'),
 ('助工', '主管'),
 ('中环', '众合'),
 ('附中', '服装'),
 ('试点', '时代'),
 ('建设', '坚守'),
 ('资源', '自营'),
 ('置业', '制药'),
 ('程', '长'),
 ('知道', '指导'),
 ('帐', '账'),
 ('之一', '质押'),
 ('帐', '账'),
 ('育', '养'),
 ('服装', '负责'),
 ('停止停', '听证听'),
 ('止', '证'),
 ('先进', '现今'),
 ('负责', '副总'),
 ('建设', '减少'),
 ('时限', '事项'),
 ('检查', '建成'),
 ('符合', '孵化'),
 ('相应', '享有'),
 ('放假', '房价'),
 ('同志', '通知'),
 ('储', '船'),
 ('以及', '移交'),
 ('统筹', '通车'),
 ('以', '有'),
 ('以', '有'),
 ('以金', '有竞'),
 ('成', '长'),
 ('面英', '棉影'),
 ('雄战', '像众'),
 ('公司', '工商'),
 ('公司流', '供水料'),
 ('吴江', '武进'),
 ('事项', '实现'),
 ('事项', '实行'),
 ('直流', '制冷'),
 ('衍生', '延伸'),
 ('供应', '共用')]